## Introduction

In this notebook, we will demonstrate how to use the [`censusdis`](https://github.com/vengroff/censusdis) package 
to download some US Census data and then how to use the [`divintseg`](https://github.com/vengroff/divintseg) package 
to compute some diversity and integration metrics.

In this example, we will look at US Census redistricting data from the towns of South Orange
and Maplewood (collectively known as SoMa) in Essex County,
NJ. We chose redistricting data because it has the demographic data we are interested in
studying, including race and ethnicity.

Once you are familiar with the API and how to use it, you can easily experiment with
similar analysis of the area where you live.

## Environment Setup

We assume you are running this notebook in an environment where the necessary 
packages have been `pip` installed. You should be able to get everything you
need with just 

```
pip install censusdis
```

which will also install `divintseg` and various other dependencies.

## Imports

We need to import out resistricting data API, some utilities for getting map data, and the 
[`divintseg`](https://github.com/vengroff/divintseg) 
package that computes diversity and integration metrics, along with `pandas` for some basic data
manipulation.

In [1]:
import censusdis.redistricting as crd

from censusdis.states import STATE_NJ
from censusdis.maps import ShapeReader

import divintseg as dis

import pandas as pd

## US Census API Key

The US Census API uses a key to identify callers. If you don't already have a key, you can request
one [here](https://api.census.gov/data/key_signup.html). Please put your key into this cell before 
running the notebook.

For small queries like in this demo notebook, the API seems to work without a key, so you can leave
it set to `None`, but for more serious work you will want to obtain a key.

In [2]:
CENSUS_API_KEY = None

## Basic Configuration

### Year

We can choose which census year we want to look at, 2010 or 2020.

In [3]:
YEAR = 2020

### Field Group

A field group is a set of fields that the US Census uses to provide data
in the various data sets it publishes. These groups cover all kinds of
topics. We are interested in demographics and we are going to be using
redistricting data, so the field groups available are those summarized 
[here](https://api.census.gov/data/2020/dec/pl/groups.html). Don't worry
if nothing on that page means anything to you right now. We'll explain it
here.

If we choose P1, then the data is grouped purely based on race, 
not taking ethnicity into account at all. If we choose P2, then the data is
first grouped by ethnicity, with people reporting Hispanic or Latino ethinicity
put into one group regardless of their race. Everyone else is then divided into
groups based on their race.

Thus, P2 has one group that P1 does not have, which is Hispanic or Latino of 
any race. In the P1 data set, people who are in the Hispanic or Latino group 
in P2 are instead classified into one of the race-based groups. 

For more information, including additional options P3 and P4, see this
additional 
[documentation](https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html).

In [4]:
FIELD_GROUP = "P2"

### Fields

Since the specific fields that exist within a field group vary by year,
we have to make a metadata query to find out what they are. This is a
simple one-liner.

In [5]:
fields, total_field, fields_by_race = crd.metadata(YEAR, FIELD_GROUP)

Each of the fields has a name (in `field_names`), and the API also groups them into 
racial groups (in `fields by race`) in case we want to use that for additional 
analysis. 

For the work we are doing here, we are just going to pass some of this metadata
on to our data query. But we will take a quick look just to see the richness of the
data provided by the census for people who are multiracial.

In [6]:
fields

{'P2_070N': 'White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
 'P2_071N': 'Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
 'P2_073N': 'White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
 'P2_060N': 'Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
 'P2_030N': 'White; Black or African American; Asian',
 'P2_054N': 'White; Black or African American; Asian; Some Other Race',
 'P2_042N': 'Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
 'P2_066N': 'White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
 'P2_043N': 'Black or African American; Asian; Some Other Race',
 'P2_031N': 'White; Black or African

## South Orange and Maplewood Tracts in Essex County, NJ

We want to take a look at data from the towns of South Orange
and Maplewood (collectively known as SoMa) in Essex County,
NJ. 

### Essex County

For Essex County, we found the code `'013`' on this 
[wikipedia page](https://en.wikipedia.org/wiki/List_of_counties_in_New_Jersey).

In [7]:
COUNTY_ESSEX_NJ = "013"

### SoMa Tracts

We found the tracts that make up the two towns by looking
at 
[this map](https://www2.census.gov/geo/maps/dc10map/tract/st34_nj/c34013_essex/DC10CT_C34013_002.pdf).
We format them as strings using the convention of
Census API, which is a six-digit string.

In [8]:
tracts_soma = [f"{t:06}" for t in range(19000, 20000, 100)]
tracts_soma

['019000',
 '019100',
 '019200',
 '019300',
 '019400',
 '019500',
 '019600',
 '019700',
 '019800',
 '019900']

### SoMa Data Query

Now we can query the data. Lets look at the arguments to our call one by one:

- `STATE_NJ` - this is the state we are interested in.
- `YEAR`- the year we want data for. For the redistricting API, 2000, 2010, and 2020 are the three valid option.
- `block` - the resolution of data we want. We want a row for each block in SoMa. 
- `field_names.keys()` - these are all the fields we want data for. The available fields vary by year and group, which
   is why we made the `crd.metadata` call above to get them.
- `county=COUNTY_ESSEX_NJ` - this is a filter. We only want data for Essex County.
- `tract=tracts_soma` - this is a second filter saying that within the county we only want the specificed tracts.
- `key=CENSUS_API_KEY` - our API key.

The return value will be a `pd.DataFrame` containing a row for each block (the resolution we specified). In order to make
analysis of diversity and integration at various levels of geographic aggregation easier (e.g. using the 
[divintseg](https://github.com/vengroff/divintseg) package) the identifiers of all of the nested geographies from the 
state down to the block are included in each row. In this case that means we have columns for STATE, COUNTY, TRACT, BLOCK_GROUP,
and BLOCK. After these columns, we have one column for each of the demographic fields we asked for.

In [ ]:
df_soma = crd.data(
    STATE_NJ,
    YEAR,
    "block",
    fields.keys(),
    county=COUNTY_ESSEX_NJ,
    tract=tracts_soma,
    key=CENSUS_API_KEY,
)

In [ ]:
df_soma

## Compute Diversity and Integration

Now that we have the census data telling us how many people of each group there are
in each block of SoMa, we can calculate diversity and inclusion at the tract over
block level. For a detailed explination of what we are actually calculating here, see
the [README.md](https://github.com/vengroff/divintseg/blob/main/README.md) in the 
[divintseg package](https://github.com/vengroff/divintseg).

In [ ]:
df_soma_dis = dis.di(df_soma, by="TRACT", over="BLOCK", drop_non_numeric=True)
df_soma_dis

## Plot on a Map

Tabular data is great, but for geographic data we often want to plot things on a map. In order
to do that, we have to load the outlines of the tracts we want to plot. These come in files
called shapefiles, and we have a convenient API to load them.

### Load shapefiles or used cached versions

In order to plot the data on a map, we need some shapefiles that
contain the boundaries of the counties and states we want to plot. The API
we will use will download the shapefiles from the US Census and cache 
them in a local directory pointed to by `SHAPEFILE_ROOT`.

If you want to browse available file you can look at
[this site](https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=Census+Tracts).

The specific zip file the code will download for us is the NJ census tract
boundary file
[`tl_2020_34_tract.zip`](https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_34_tract.zip).

In [ ]:
import os
import os.path

SHAPEFILE_ROOT = os.path.join(os.environ["HOME"], "data", "shapefiles")

# Make sure it is there.
if not os.path.isdir(SHAPEFILE_ROOT):
    os.mkdir(SHAPEFILE_ROOT)

### Load the Shapefile for NJ Tracts

Now that we have the shapefile we need, we can load it into our notebook.
It will be loaded into a `GeoDataFrame`, which is very much like an 
ordinary `DataFrame` that you are used to, except that it has a special
`geometry` column that defines a geometric shape for each row. In this 
case the geometry is the outline of each census tract in the state of NJ.

In [ ]:
reader = ShapeReader(SHAPEFILE_ROOT, year=YEAR)

In [ ]:
gdf_nj_tracts = reader.read_shapefile(STATE_NJ, "tract")
gdf_nj_tracts.head()

###  Merge in our Diversity and Integration Data

Now that we have the geometry of each census tract, we can select just
the ones for Essex County and then merge in the diversity and integration
metric we computed so we have everything we need to make our plots in one 
`GeoDataFrame`.

In [ ]:
gdf_essex_di = gdf_nj_tracts[gdf_nj_tracts["COUNTYFP"] == COUNTY_ESSEX_NJ].merge(
    df_soma_dis, left_on="TRACTCE", right_on="TRACT"
)[["TRACTCE", "diversity", "integration", "geometry"]]

In [ ]:
gdf_essex_di.head()

### Plot the Maps

Now we can plot diversity and integration as heat maps. We did just some 
very basic styling on these plots, but of course you can do whatever you
want to make them the best visualizions for your purposes.

In [ ]:
ax = gdf_essex_di.plot(
    "diversity",
    cmap="Greens",
    edgecolor="black",
    legend=True,
    vmin=0.0,
    vmax=1.0,
)

ax.set_title("Diversity in SoMa Tracts")

ax.tick_params(
    left=False,
    right=False,
    bottom=False,
    labelleft=False,
    labelbottom=False,
)

In [ ]:
ax = gdf_essex_di.plot(
    "integration",
    cmap="Greens",
    edgecolor="black",
    legend=True,
    vmin=0.0,
    vmax=1.0,
)

ax.set_title("Integration in SoMa Tracts")

ax.tick_params(
    left=False,
    right=False,
    bottom=False,
    labelleft=False,
    labelbottom=False,
)